In [8]:
# TODO FIX ERRORS
# TODO DATA TO MONGO DB

import pandas as pd

# https://www.kaggle.com/datasets/kazanova/sentiment140?resource=download
# https://www.youtube.com/watch?v=KgrdifrlDxg&t=939s&ab_channel=SebastianRaschka
# https://github.com/bentrevett/pytorch-sentiment-analysis

tweetsDF = pd.read_csv("training.1600000.processed.noemoticon.csv", engine="python", header=None, encoding='latin-1')


tweetsDF.columns = ["sentiment", "ids", "date", "flag", "user", "text"]

tweetsDF = tweetsDF[["sentiment", "text"]]
tweetsDF.loc[tweetsDF["sentiment"] == "1"]

,sentiment,text


In [3]:
from torchtext import data

LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', lower=True)

fields = [('score', None), ('id', None), ('date', None), ('query', None), ('name', None), ('tweet', TWEET), ('category', None), ('label', LABEL)]

twitterDataset = data.TabularDataset(
    path="training.1600000.processed.noemoticon.csv"
    ,format="CSV"
    ,fields=fields
    ,skip_header=False
)

(train, test, valid) = twitterDataset.split(split_ratio=[0.8, 0.1, 0.1])

vocab_size = 20002
TWEET.build_vocab(train, max_size = vocab_size)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test)
    , batch_size = 32
)

AttributeError: module 'torchtext.data' has no attribute 'LabelField'

In [4]:
import torch.nn as nn 

class LSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden, _) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = LSTM(100, 300, 20002)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs +1):

        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            opt.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict, batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.items() * batch.tweet.size(0)
        training_loss /= len(train_iterator)

        model.eval()
        for batch_idx, batcj in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict, batch.label)
            valid_loss += loss.data.items() * x.size()

        valid_loss /= len(valid_iterator)
        print("Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}".format(epoch, training_loss, valid_loss))